# Convert a spreadsheet to a website

Convert a Google sheet to a website, where each `col-/row-` is a class on the HTML element.



## Introduction



### A couple of rules

Name your pages like ```App layout``` or ```Shop layout```, with an optional ```Shop data``` which will automatically load as a data table to be used in the template like ```{{#shop-data}}{{description}}{{/shop-data}}```. This would repeat for every row in the data table printing out all of the description columns. Data tables should list the column names as the first row. Hiding sheets doesn't matter to the converter but makes it easier to manage lots of pages or see only the one's you are working on.

Use HTML or Mustache or Markdown in your sheets. Rows are automatically converted to top level ```<div>``` elements. The class ```col-2``` is added to count the number of columns. Cells are automatically converted to ```<div>``` as children on the row ```<div>```. The class ```cell-0``` is automatically added as the index of the cell, not including any variables in between.

Beginning a cell with ":" comma creates variables and the cell right of a variable is used as the value. Variables do not affect column count or HTML output. Vairables can be used just like mustache variables, e.g. `{{variable}}`. Variables are combined with subpages, that is, any variable set on one page can be used page another page loaded by a URL, ```{{> section}}```, or ```::render```.

Special variables are:

`::stylesheet`, load css links.

`::script`, load javascript links.

```:logo```, which shows up in the favicon.

```:title``` is used as page title.

```::render``` forces another template to render whether or not there is a link to it, this is a list and can be used as many times as you want.

```{{> section}}``` inserts a {{#section}}{{/section}} mustache template, combines section from mustach and render from this framework. Pages that are not linked to, rendered, or sectioned, won't be scanned at all. Sections should be refered to in lowercase with spaces converted to dashes and without the word ```layout``` attached, i.e. ```{{> shop-menu}}```.

**Now for the complicated part!** ```/groups/1``` will load the layout ```groups``` and filter by the ```url``` or ```link``` column to match a single group. If no ```link``` or ```url``` column matches, it will search the column named after the page, the ```groups``` column will be used to match data categorically, all data rows matching ```/group/1``` will be available for repetion with a ```{{#groups-data}}{{/groups-data}}``` section tag.

The same rule applies to section includes, but even better! You can include a filtered section from the parents match, if you had groups and subgroups for each of the groups. ```{{> subgroup/groups}}``` will use the ```{{groups}}``` variable from the current page, and filter subgroups accordingly. Any subgroup with the subgroup matching ```/group/1``` will be available for repetition using ```{{#subgroup-data}}{{/subgroup-data}}```.

URLs such as ```/link``` inherently forces the spreadsheet page named Link layout, or Link data to render. Links to other pages are automatically detected. Names of pages are automatically converted to lowercase, with dashes instead of spaces.

Use ```::variable``` to create a list such as ```::stylesheet``` or ```:scripts```.

[Markdown description at markdownguide.org](https://www.markdownguide.org/basic-syntax)

[Mustache guide from NodeJs](https://github.com/janl/mustache.js)


### TODO

Convert a slides document to a website in similar fashion.

Inlining all styles and scripts and images as data-uri for faster load times.

Automatically detecting which features to add such as ```Buy Now``` buttons and contact us chats.

readme.md?



In [ ]:
// placeholder for readme


## API entry points

Basically 3, check if a sheet has been imported, import the sheet, and setup a backend.

Copy marketing is an entry point for copying the marketing template and then inviting the user to edit it.



### check if a sheet is imported



#### the code

get sheet purchases?

google sheet handler?

get sheet identifier from link?



In [ ]:
var util = require('util');
var uuid = require('uuid/v1');
var importer = require('../Core');
var getDataSheet = importer.import('google sheet array objects');
var getInfo = importer.import('get google sheet info');
var addRow = importer.import('add row data google sheet');
var updateRow = importer.import('update a row in google sheets')

var purchaseId = '1kWjkjLGxQyzFUzRLBk3LpcjPW3UjcaF-PBMDX_3hZfM';
var project = 'spahaha-ea443';

var isInvalidDomain = (match, domain) =>
    !match || domain && domain !== match.domain && domain !== match.bucket

function safeName(name) {
    return name.replace(/[^a-z0-9\-]/ig, '-').substr(0, 40).toLowerCase();
}

async function addSheet(docId, title, email) {
    var name = safeName(title.replace(/\s*(configuration|config)\s*/ig, ''))
        + '-' + uuid().substr(0, 5);
    return await addRow(purchaseId, {
        timestamp: Date.now(),
        name: title,
        email: email || '',
        address: '',
        domain: '',
        bucket: name + '.sheet-to-web.com',
        project: project,
        sheet: docId
    }, 'Purchases').then(() => name + '.sheet-to-web.com')
}

async function getSheet(link, domain, email) {
    var title, docId;
    var info = await getInfo(link)
    title = info.properties.title;
    docId = info.spreadsheetId;
    var purchases = (await getDataSheet(purchaseId, 'Purchases'))
    var match = purchases.filter(p => p.sheet == docId)[0]
    if(domain && isInvalidDomain(match, domain)) {
        throw new Error(`sheet ${docId} doesn't match domain ${domain}`)
    } else if(!match) {
        return await addSheet(docId, title, email);
    }
    console.log(`Purchase ${docId} already exists: ${match.domain} or ${match.bucket}`);
    return await updateRow(purchaseId, r => r.sheet == docId, Object.assign(match, {
        name: title,
        email
    }), 'Purchases');
}

module.exports = getSheet;

### import the sheet in to a bucket



#### the code

sheet marketing import handler?

sheet marketing import?


In [ ]:
var importer = require('../Core');
var getSheet = importer.import('get sheet purchases');
var getTemplates = importer.import('templates google sheet');
var wrapTemplate = importer.import('output google sheet template');
var getTemplateProperties = importer.import('google sheet template properties');
var collectTemplateResources = importer.import('collect google sheet resources');
var getTemplateByUrl = importer.import('convert sheet helper functions');

async function importSheet(link, domain) {
    var properties = {}, templates, key
    var match = await getSheet(link, domain)
    console.log(match)
    domain = domain || match.bucket
    templates = await getTemplates(match.sheet)
    key = getTemplateByUrl(templates, '/')
    await getTemplateProperties(key, properties, templates)
    var page = await wrapTemplate('', key, properties[key], properties)
    var resources = (await collectTemplateResources(key, page, properties, templates, domain))
        .filter((cur, i, arr) => arr.indexOf(cur) == i)
        .filter(r => !r.includes(':') && r.includes('.'))
    return resources
}

module.exports = importSheet;


## Template functions

Filtering a sheet's data based on the URL.



### Filter sheet data



#### the code

filter data sheet based on url?


In [ ]:
var TRIM = /^\s*\/\s*|\s*\/\s*$/ig;

var compareLink = (dataValue, base, link) => {
    var compareOps = [
        link,
        base + '/' + link,
        link.split('/').slice(1).join('/'),
        base,
        link.substr(0, (dataValue || '').length)
    ]
    var result = false
    compareOps.forEach(comp => {
        if(dataValue && comp.replace(TRIM, '') === dataValue.replace(TRIM, ''))
            result = comp.replace(TRIM, '')
        return false
    })
    return result
}

function unfilteredData(key) {
    console.log(`rendering unfiltered ${key}`) 
    return (val, render) => render(`{{#${key}-original-data}}${val}{{/${key}-original-data}}`)
}

function filteredData(key, match, properties, categorical) {
    return function(val, render) {
        var link = (render(`{{{${match}}}}`) || properties[match] || '').replace(TRIM, '');
        var base = (render(`{{{base}}}`) || properties['base'] || '').replace(TRIM, '');
        if(link.substr(0, base.length) === base) {
            link = link.substr(base.length).replace(TRIM, '');
        }
        var matchKey = key + '-filtered-data';
        var restore = properties[key + '-original-data'];
        
        if(categorical) {
            // handle multiple results for use with categories
            properties[matchKey] = restore.filter(data => compareLink(data[key], base, link))
        } else {
            properties[matchKey] = restore.filter(data => compareLink(data['link'], base, link)
                                                   || compareLink(data['url'], base, link));
            if(properties[matchKey].length !== 1) {
                throw new Error(`Unique key not found: ${key} ${match} ${link} ${properties[matchKey].length}`)
            }
        }

        console.log(`rendering filtered ${categorical 
                    ? 'categorical' 
                    : 'unique'}: ${key} ${match} ${link} ${properties[matchKey].length}`);
        var rendered = render(`{{#${matchKey}}}${val}{{/${matchKey}}}`);
        delete properties[matchKey];
        return rendered;
    }
}

module.exports = {
    filteredData,
    unfilteredData
}


### get sheet properties

Make sure all variables can be loaded from the document.

Render sections recursively.


#### the code

google sheet template properties?


In [ ]:
var importer = require('../Core');
var getDataSheet = importer.import('google sheet array objects');
var renderRows = importer.import('google sheet layout template');
var getRows = importer.import('get worksheet rows');
var {filteredData, unfilteredData} = importer.import('filter data sheet based on url');
var promiseOrResolve = importer.import('resolve promise property');

var isWrapper = rows => rows.length === 1 && rows[0].length === 1
              && rows[0][0].match(/\{\{\s*>\s*(.*?)(\/(.*?-)+link)*\}\}/ig);

var isFiltered = (url) => url.split('/').length > 1

// TODO: remove previous properties if for some strange reason recalling the same template,
//   prevents situations like ::stylesheet being added to many times
var addedProperties = {};

function getTemplateProperties(key, properties, templates) {
    if(typeof templates[key] === 'undefined') {
        throw new Error(`section "${key}" not found!`)
    }
    // load template data
    var rows;
    return promiseOrResolve(templates[key].data, 'rows', getDataSheet)
        .then(data => {
            properties[key + '-original-data'] = data;
            properties[key + '-data'] = unfilteredData.bind(null, key);
        })
        // load template layout
        .then(() => promiseOrResolve(templates[key].template, 'rows', getRows))
        .then(rs => {
            rows = rs || []
            return rows.flat()
              .reduce((p, c, j) => p.then(() => matchSections(c, properties, templates)), Promise.resolve())
        })
        // detect if this is just a wrapper template and don't render rows, do a direct replacement
        .then(() => isWrapper(rows) ? rows[0][0] : renderRows(key, rows, properties, templates))
        .then(template => (properties[key] = template))
}

// must do this up front so we can process all data
var matchSections = (cell, properties, templates) => importer
    .regexToArray(/\{\{\s*>\s*(.*?)\s*\}\}/ig, cell, 1)
    .reduce((promise, section) => promise
        .then(() => getTemplateProperties(section.split('/')[0], properties, templates))
        .then(() => isFiltered(section)
              ? createAssignFilter(section, properties)
              : Promise.resolve()),
            Promise.resolve())

var isCategorical = (data, key) => data.filter(row => row.hasOwnProperty(key)).length > 0;

function createAssignFilter(section, properties) {
    var key = section.split('/')[0];
    var match = section.split('/').slice(1).join('/');
    properties[section] = properties[key];
    
    // add a special partial for the filtered data
    var categorical = isCategorical(properties[key + '-original-data'] || [], key)
    // automatically wrap unique templates in a data section for accessing filtered properties
    if(!categorical && !properties[key].includes(`{{#${key}-data}}`)
        && typeof properties[key + '-original-data'] !== 'undefined') {
        properties[section] = `{{#${key}-data}}${properties[key]}{{/${key}-data}}`;
    }
    // run the filtered function instead of using the array
    properties[key + '-data'] = filteredData.bind(null, key, match, properties, categorical);
}

module.exports = getTemplateProperties;


### create `col-/row-` sections

Convert a google sheet page to an HTML template



#### the code

google sheet layout template?



In [ ]:

function safeName(name) {
    return name.replace(/\//ig, ' ').replace(/[^a-z0-9\- ]/ig, '-').substr(0, 40);
}

function escape(s) {
    return s.replace(/[-\/\\^$*+?.()|[\]{}]/g, '\\$&');
}

function getDataClasses(c, data) {
    // get classes from mustache vars used with supplied data
    return typeof data != 'object' ? [] : (data || [])
        .reduce((keys, cur) => keys.concat(Object.keys(cur)), [])
        .filter((k, h, a) => a.indexOf(k) == h)
        .filter(k => c.match(new RegExp(`\\{\\{\\s*[>#\\/]?\\s*${escape(k)}\\s*\\}\\}`, 'ig')))
}

function defineProperty(c, value, properties) {
    if (c.substr(0, 2) === '::' || c === ':render') {
        if(c === ':render') c = '::render'; // just to fix using it below
        if(typeof properties[c.substr(2)] == 'undefined') {
            properties[c.substr(2)] = [];
        } else if (!Array.isArray(properties[c.substr(2)])) {
            properties[c.substr(2)] = [properties[c.substr(2)]];
        }
        properties[c.substr(2)][properties[c.substr(2)].length] = value;
    } else {
        properties[c.substr(1)] = value;
    }
}

function renderRows(key, rows, properties, templates) {
    // set object properties for mustache template
    var html = (rows || []).reduce((arr, row, i) => {
        var rowsHtml = row.reduce((arr, c, j) => {
            if(c.substr(0, 1) === ':') {
                // use subsequent column for property values
                defineProperty(c, row[j + 1], properties)
                
            // render if it is not the value for the previous property
            } else if(j === 0 || row[j - 1] && row[j - 1].substr(0, 1) !== ':') {
                var dataClasses = getDataClasses(c, properties[key + '-original-data'])
                    .map(k => 'val-' + safeName(k))
                    .join(' ')
                var sectionClasses = getDataClasses(c, [properties, templates])
                    .map(k => 'section-' + safeName(k))
                    .join(' ')
                arr[arr.length] = `
<div class="cell-${arr.length} ${dataClasses} ${sectionClasses}">
${c}
</div>
`;
            }
            return arr;
        }, []);

        if(rowsHtml.length > 0) {
            arr[arr.length] = `
<div class="row-${arr.length} ${properties['class'] || ''} col-${rowsHtml.length}">
${rowsHtml.join('')}
</div>
`;
        }

        return arr;
    }, []);

    return html.join('');
}

module.exports = renderRows;



### Output page template

Save the generated template to an HTML file, wrapping it in a base template


#### the code

output google sheet template?


In [ ]:
var fs = require('fs');
var path = require('path');
var Mustache = require('mustache');

function safeName(val, render) {
    return render(val).replace(/[^a-z0-9\-]/ig, '-').substr(0, 40)
}

function toJSON(val, render) {
    return render(JSON.stringify(val))
}

function segment(url, val, render) {
    return url.split('/')[render(val)]
}

function wrapTemplate(path, key, html, properties) {
    properties['safeName'] = () => safeName
    properties['toJSON'] = () => toJSON
    properties['segment'] = () => segment.bind(null, path)
    var classNames = path.replace(/\//ig, ' ')
        + ' ' + (key !== path.split('/')[0] ? key : '')
        + ' ' + (properties['class'] || '')

    var domain = '';
    if(typeof properties['domain'] != 'undefined') {
        domain = properties['domain'].includes(':') ? '' : 'https://{{domain}}';
    }
    
    // automatically set title if it isn't set manually
    var result;
    if(typeof properties['title'] == 'undefined' && (result = (/<h1>(.*)<\/h1>/ig).exec(html))) {
        properties['title'] = result[1];
    }
    
    var pageHtml = `
<!doctype html>
<html>
<head>
<meta charset="utf-8">
<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1.0, user-scalable=no">
<link rel="icon" href="{{logo}}">
{{#base}}<base href="/{{.}}" />{{/base}}
<meta property="og:type" content="website">
<meta property="og:title" content="{{title}}">
<link rel="canonical" href="${domain}/${path}">
<title>{{title}}</title>
{{#banner}}
<style>
body > div.col-1:nth-of-type(2):before {background-image: url('{{{.}}}');}
body > div.col-1:nth-of-type(2):after {background-image: url('{{{.}}}');}
</style>
{{/banner}}
{{#stylesheet}}<link rel="stylesheet" href="{{.}}">{{/stylesheet}}
</head>
<body class="${classNames}">
${html}
{{#script}}<script async src="{{.}}"></script>{{/script}}
</body>
</html>`;

    Mustache.parse(pageHtml);
    // use properties for view and for partials
    return Mustache.render(pageHtml, properties, properties);
}

module.exports = wrapTemplate;



## Automatic HTML router

TODO: make this a utility for selenium web crawler.


### match routes to templates


#### the code

find known routes to sheets?


In [ ]:
var importer = require('../Core')
var wrapTemplate = importer.import('output google sheet template')
var getTemplateProperties = importer.import('google sheet template properties')
var getTemplateByUrl = importer.import('convert sheet helper functions')

var TRIM = /^\s*\/\s*|\s*\/\s*$/ig;

var isFiltered = (url) => url.split('/').length > 1

// combine with "getSections" by using fake "{{> url/path}}" include
function collectRoutes(routes, properties, templates, rendered) {
    var local = routes.concat(properties['render'] || [])
        .filter((link, i, arr) => !rendered.includes(link)
                // protocol means it's absolute remote path and not to try to generate it
                && arr.indexOf(link) == i && !link.includes(':'))

    local.forEach(link => rendered[rendered.length] = link)
    
    var promises = local
        // promise in series so there is no data collisions
        .map(link => resolve => {
            
            var trimmedBase = (properties['base'] || '').replace(TRIM, '')
            link = link.replace(/#.*$/, '') // remove hash #
            link = link.replace(TRIM, '') // remove extra slashes
            
            if(link.substr(0, trimmedBase.length) === trimmedBase) {
                link = link.substr(trimmedBase.length).replace(TRIM, '');
            }
            // any part of a path can contain the reference to a page template
            var key = getTemplateByUrl(templates, link);
            var newProps = Object.assign({}, properties);
        
            // create a temporary template to filter by
            newProps[key + '-' + key + '-link'] = link;
            templates[key + '-' + key] = {template: {rows: [[
                isFiltered(link)
                    ? `{{> ${key}/${key}-${key}-link}}`
                    : `{{> ${key}}}`
            ]]}}
        
            return getTemplateProperties(key + '-' + key, newProps, templates)
                .then(() => wrapTemplate(link, key, newProps[key + '-' + key], newProps))
                .then(page => {
                    var pages = {};
                    pages[link] = page;
                    resolve(pages)
                })
        })
    
    return importer.runAllPromises(promises)
        .then(results => results.reduce((obj, r) => Object.assign(obj, r), {}))
}

module.exports = collectRoutes;



### find all external resources

TODO: preload specific items like:

- first image could be down-encoded to a data-uri,
- stylesheets should be under a certain size and embedded as styles.


#### the code

collect external content and resources?


In [ ]:
var {Readable} = require('stream');
var importer = require('../Core');
var {Remarkable} = require('remarkable');
var md = new Remarkable({html: true, xhtmlOut: true, breaks: true});
var importer = require('../Core');
var {selectDom} = importer.import('select tree');

var TRIM_ENTITIES = /<\/?\s*p\s*>|[\s\n\r]+|<\s*br\s*\/?\s*>/ig

function safeName(name) {
    return name.replace(/[^a-z0-9\-]/ig, '-').substr(0, 40);
}

function collectExternalResources(page, rendered, routes) {
    // get all images and urls from template
    var body = selectDom('*', page);
    
    // replace all leaf Text nodes with markdown
    var textObjs = selectDom([`//*[not(self::STYLE) and contains(., /text())]`], body);
    textObjs.forEach((parent, i) => {
        parent.childNodes.forEach(string => {
            if(string.nodeType !== 3) return;
            var mdHtml = md.render(string.textContent);
            // if all markdown did was insert a paragraph and line break, use value instead
            if(mdHtml.replace(TRIM_ENTITIES, '').trim()
               != string.textContent.replace(TRIM_ENTITIES, '').trim()) {
                string.replaceWith.apply(string, selectDom(['//BODY/*'], `<html><body>${mdHtml}</body></html>`));
            }
        })
    })
    
    // add IDs to h1, h2, h3, etc elements that match their text contents
    var headingsObjs = selectDom(['(//h1|//h2|//h3|//h4)[not(@id) and not(./ancestor::nav)]'], body);
    headingsObjs.forEach(h => h.setAttribute('id', safeName(h.textContent)));
    
    var paragraphs = selectDom(['//p'], body)
    paragraphs.forEach(p => {
        p.setAttribute('class', selectDom(['.//*'], p)
                       .map(e => e.tagName).join(' '))
        var img = selectDom(['.//img'], p)[0]
        var id = 'id' + safeName(p.textContent || img.src)
        p.setAttribute('id', id)
        if(img) {
            var style = img.ownerDocument.createElement('style')
            var src = decodeURIComponent(img.getAttribute('src'))
                .replace(/(-final)*\.[^\.]*$/ig, '')
            style.appendChild(img.ownerDocument.createTextNode(`
#${id}:before {background-image: url("${src}-final.jpg");}`));
            p.parentNode.insertBefore(style, p)
        }
    })
    
    var linksObjs = selectDom(['//a[@href]'], body);
    var links = linksObjs.map(l => decodeURIComponent(l.getAttribute('href')));
    
    // TODO: convert images and add timestamps, add svg
    var imgObjs = selectDom(['//img[@src]'], body);
    var imgs = imgObjs.map(l => decodeURIComponent(l.getAttribute('src')));
    imgObjs.forEach(img => {
        var src = decodeURIComponent(img.getAttribute('src'))
            .replace(/(-final)*\.[^\.]*$/ig, '-final.jpg')
        img.setAttribute('src', src)
    })

    // TODO: scan for urls and inline
    var stylesObjs = selectDom(['//link[@href]'], body);
    var styles = stylesObjs.map(l => l.getAttribute('href'));

    // TODO: add timestamps and inline
    var scriptsObjs = selectDom(['//script[@src]'], body);
    var scripts = scriptsObjs.map(l => l.getAttribute('src'));
    
    // TODO: add CSS imports
    var backgrounds = importer.regexToArray(/url\(['"]?(.*?)['"]?\)/ig, page, 1);
    
    var searches = [].concat.apply([], [
        imgs, styles, backgrounds, scripts
    ])
    
    links.forEach(s => routes[routes.length] = s)
    searches.forEach(s => rendered[rendered.length] = s)
    
    // TODO: copy resource images to output directory
    var newPage = body.ownerDocument.documentElement.outerHTML
    backgrounds.forEach(b => newPage = newPage.replace(b, b.replace(/(-final)*\.[^\.]*$/ig, '-final.jpg')))
    var stream = new Readable();
    stream.push(newPage);
    stream.push(null);
    return Promise.resolve(stream);
}

module.exports = collectExternalResources;


### collect sheet resources

Recursively load sheets and pages from URLs until all pages are generated.



#### the code

collect google sheets resources?


In [ ]:
var importer = require('../Core');
if((process.env.ENVIRONMENT || '').includes('LOCAL')
   || (process.env.ENVIRONMENT || '').includes('TEST')
   || !(process.env.ENVIRONMENT || '').includes('DEPLOY')) {
 }
var copyFileBucket = importer.import('copy file storage bucket');
var collectExternalResources = importer.import('collect external content and resources');
var collectRoutes = importer.import('find known routes to sheets');
var getTemplateByUrl = importer.import('convert sheet helper functions');

var timestamp = (new Date()).getTime();

// detect links and write out every part of the site
function collectTemplateResources(path, page, properties, templates, bucketName, rendered) {
    var streamToGoogle
    if(!(process.env.ENVIRONMENT || '').includes('DEPLOY')) {
        streamToGoogle = importer.import('test stream to output');
    } else {
        streamToGoogle = importer.import('upload files google cloud');
    }

    if(!rendered) {
        rendered = [];
    }

    // if it is the first page in the template, rename it to index.html
    if((path.split('/').length < 2 || path === '/' || path === '')
       && getTemplateByUrl(templates, path) === getTemplateByUrl(templates, '/')) {
        console.log(`using ${path} as index.html`);
        path = 'index';
    }
    
    var trimmedBase = (properties['base'] || '').replace(/^\/|\/$/ig, '');
    if(path.substr(0, trimmedBase.length) !== trimmedBase) {
        path = trimmedBase + '/' + path;
    }

    // TODO: add timestamps to generated content
    // TODO: set different permissions on files streamed to google.
    
    var routes = [];
    return collectExternalResources(page, rendered, routes)
        .then(stream => (console.log(`emitting ${path}`), stream))
        .then(stream => streamToGoogle(path + '.html', bucketName, stream, {
            contentType: 'text/html; charset=utf-8'
        } /* TODO: insert permission settings for user directory */))
        .then(() => !(process.env.ENVIRONMENT || '').includes('DEPLOY')
              ? {}
              : copyFileBucket(bucketName, path + '.html'))
        .then(() => collectRoutes(routes, properties, templates, rendered))
        .then(pages => importer.runAllPromises(Object.keys(pages).map(fileName => resolve => 
            collectTemplateResources(fileName, pages[fileName], properties, templates, bucketName, rendered)
                .then(() => resolve()))))
        .then(() => rendered)
}

module.exports = collectTemplateResources;


### test google sheets resources?

sheet to web?

In [ ]:
var fs = require('fs');
var path = require('path');
var importer = require('../Core');
var getEnvironment = importer.import('get environment');
var importSheet = importer.import('sheet marketing import');
var {glob} = importer.import('glob files')
var streamToGoogle

function initSync(env) {
    getEnvironment(env || process.env.ENVIRONMENT || 'TEST_SPREADSHEET');
    if((process.env.ENVIRONMENT || '').includes('LOCAL')
    || (process.env.ENVIRONMENT || '').includes('TEST')
    || !(process.env.ENVIRONMENT || '').includes('DEPLOY')) {
        streamToGoogle = importer.import('test stream to output');
    } else {
        streamToGoogle = importer.import('upload files google cloud');
    }
}

initSync(process.env.ENVIRONMENT);

var copyAllFiles = (resources, bucketName) => Promise.all(resources
    .map(f => f && streamToGoogle(path.basename(f),
                                  bucketName,
                                  fs.createReadStream(f))
        .catch(e => console.error(e))))

async function importTest(link, domain, env) {
    if(env) {
        initSync(env)
    }
    var resources = await importSheet(link, domain)
    // TODO: fix google storage check
    if(process.env.PROJECT_OUTPUT) {
        var existingFiles = glob(resources
            .map(r => `**/*${path.basename(r)}*`), process.env.PROJECT_OUTPUT)
        resources = resources.filter(r => existingFiles.filter(e => e.includes(path.basename(r))).length == 0)
        console.log(resources)
    }
    var newFiles = glob(resources
        .map(r => `**/*${path.basename(r)}*`), process.env.DOWNLOAD_PATH)
    resources = resources.filter(r => newFiles.filter(e => e.includes(path.basename(r))).length == 0)
    var desktopFiles = glob(resources
        .map(r => `**/*${path.basename(r)}*`), path.join(process.env.DOWNLOAD_PATH, '../Desktop'))
    var pictureFiles = glob(resources
        .map(r => `**/*${path.basename(r)}*`), path.join(process.env.DOWNLOAD_PATH, '../Pictures'))
    return copyAllFiles(newFiles.concat(desktopFiles).concat(pictureFiles), domain)
}

module.exports = importTest;



## Helper functions



### get template url

convert sheet helper functions?


In [ ]:

var getTemplateByUrl = (templates, path) => !path || path === '' || path === '/'
    ? getEntryTemplate(templates)
    : path.split('/').filter(segment => templates[segment]
                             && templates[segment].template)[0]
    || path.split('/')[0];

var getEntryTemplate = (templates) => Object.keys(templates)
    .filter(t => templates[t].template
            && templates[t].template.properties
            && templates[t].template.properties.index == 0)[0]
    || Object.keys(templates).filter(t => templates[t].template)[0];

module.exports = getTemplateByUrl;

### package.json?


In [ ]:
{
    "name": "SheetToWeb",
    "description": "Marketing site functions",
    "license": "UNLICENSED",
    "scripts": {
    },
    "engines": {
        "node": ">= 8",
        "npm": ">= 4"
    },
    "repository": {
        "type": "git",
        "url": "git+https://github.com/megamindbrian/jupytangular.git"
    },
    "dependencies": {
        "@google-cloud/compute": "^0.12.0",
        "@google-cloud/storage": "^2.5.0",
        "googleapis": "^39.2.0",
        "jsdom": "^14.0.0",
        "mustache": "^3.0.1",
        "remarkable": "^1.7.1"
    }
}
